In [1]:
# Standard libraries
import sys
import math
import random
import time
import os

# Third-party imports
import numpy as np
import pandas as pd
import pickle as pkl

import matplotlib.pyplot as plt

import tensorflow as tf

import multiprocess
import psutil

import importlib
from yapf.yapflib.yapf_api import FormatCode

# Project imports
import ephesus
import utils

In [2]:
infor = {
    'curve_injec': False, 
    'curve_type': None,
    'cut_times': None,
    'cut_trans': None,
    'cut_numbe': None,
    'cut_start_index': None,
    'detre_time_scale': None,
    'detre_type': None,
    'eb': None,
    'file_name': None,
    'injec_times': None,
    'inter_type': None,
    'initi_paddi': None,
    'linea_limb_darke_coeff': None,
    'max_ampli': None,
    'model_file': None,
    'moon_densi': None,
    'moon_eccen': None,
    'moon_epoch': None,
    'moon_incli': None,
    'moon_mass': None,
    'moon_numbe': None,
    'moon_perio': None,
    'moon_radiu': None,
    'moon_sin_w': None,
    'moon_signa': None,
    'plane_cut_injec': None,
    'plane_densi': None,
    'plane_eccen': None,
    'plane_epoch': None,
    'plane_incli': None,
    'plane_mass': None,
    'plane_moon_cut_injec': None,
    'plane_numbe': None,
    'plane_perio': None,
    'plane_radiu': None,
    'plane_sin_w': None,
    'plane_signa': None,
    'plane_type': None,
    'predi': None,
    'quadr_limb_darke_coeff': None,
    'ratio_moon_plane_radiu': None,
    'ratio_moon_stell_radiu': None,
    'ratio_plane_stell_radiu': None,
    'signa': None,
    'stell_effec_tempe': None,
    'stell_magni': None,
    'stell_mass': None,
    'stell_radiu': None,
    'tic_id': None,
    'toi': None,
    'trape_trans': None,
    'type_calcu': None,
    'type_limb_darke': None,
    'type_orbit_archi': None
}

In [3]:
main_path = '/home/rfradkin/xom/'
raw_tess_path = '/scratch/data/tess/lcur/spoc/raws/'
tess_metad_path = '/data/scratch/data/tess/meta/'
xom_data_path = '/data/scratch/data/moon/'

### Import Light Curves

In [4]:
numbe_of_curve = 600

# Access the file names
files = utils.retur_secto_files(f'{raw_tess_path}')
# Choose a random sample
sampl = utils.retur_rando_sampl(numbe_of_curve, len(files))
# Create a list containing the file names
files = [files[index] for index in sampl]

In [5]:
light_curve_toi = utils.retur_fits_data(files, infor, f'{raw_tess_path}')

### Mark TOIs, EBs, and Stellar Parameters

In [6]:
# Find TIC ID of all TOIs
toi_full = pd.read_csv(f'{tess_metad_path}toi.csv')
toi_id = toi_full['TIC ID'].to_numpy()
# Mark TOIs in light curve dataset
utils.mark_TOI(light_curve_toi, toi_id)
# Remove the TOIs
light_curve = utils.remov_TOI(light_curve_toi)

In [7]:
TIC8 = pd.read_csv(f'{tess_metad_path}nomi_miss_2min_info.csv')

for i in range(len(light_curve)):
    TIC_ID = light_curve[i, -1, 1]['tic_id']
    # Find current TIC ID in TIC8
    star_infor = TIC8[TIC8.tici == TIC_ID]
    # Make sure TIC ID is in TIC8
    if len(star_infor):
        # Gather stellar radius, mass, temperature, and magnitude
        if str(star_infor.radistar.iloc[0]) != 'nan':
            light_curve[i, -1, 1]['stell_radiu'] = star_infor.radistar.iloc[0]
        if str(star_infor.massstar.iloc[0]) != 'nan':
            light_curve[i, -1, 1]['stell_mass'] = star_infor.massstar.iloc[0]
        if str(star_infor.tmptstar.iloc[0]) != 'nan':
            light_curve[i, -1, 1]['stell_effec_tempe'] = \
star_infor.tmptstar.iloc[0]
        if str(star_infor.tmag.iloc[0]) != 'nan':
            light_curve[i, -1, 1]['stell_magni'] = star_infor.tmag.iloc[0]

### Injecting

In [8]:
# Divide Curve
divid = len(light_curve) // 8
# Index of planet injections
plane_index = range(0, divid)

# Find the last injected planmoon curve if continuing an injection
# Otherwise, set it equal to the curve after the final planet injection
try:
    if conti_injec:
        pass
except:
    last_injec = plane_index[-1] + 1
    
# Index of planet and moon injections
plane_moon_index = range(last_injec, 7 * divid)

# Find the number the last injection
last_injec_numbe = max(plane_index[-1], plane_moon_index[-1]) + 1

injec_curve = np.copy(light_curve)
# Copy dictionaries
for i in range(len(injec_curve)):
    injec_curve[i, -1, 1] = injec_curve[i, -1, 1].copy()

In [9]:
# Check to see the current CPU usage
cpu_usage = 0
itera = 5000
for i in range(itera):
    cpu_usage += psutil.cpu_percent()
print(f'About {(cpu_usage / itera):.1f}% of the total CPU is in use')

About 9.8% of the total CPU is in use


In [17]:
# Save injections every curve_save_numbe curves
curve_save_numbe = 100
curve_save_index = [last_injec_numbe]

curre_save_index = 0

# Just run the whole injection if the number of injected curves
# is less than 4 times the minimum save amout
if last_injec_numbe < curve_save_numbe * 4:
    pass
else:
    while curre_save_index < last_injec_numbe:
        curve_save_index.insert(-1, curre_save_index)
        curre_save_index += curve_save_numbe

In [11]:
%%capture

# Number of cpus
cpu_number = 20

# Set up save flag
save = False

# Planet injection setup
plane_max_numbe = 1
moon_max_numbe = 0
type_orbit_archi = 'plan'

injec_argum = []

for i in plane_index:
    injec_argum.append(
        (injec_curve[i], plane_max_numbe, moon_max_numbe, type_orbit_archi))

# Planet and moon injection setup
plane_max_numbe = 1
moon_max_numbe = 1
type_orbit_archi = 'planmoon'
separ_plane_moon = False
reinj = False
anima_path = None

# Setup initial save file
if len(curve_save_index) > 2:
    with open(f'{main_path}tempo.txt', 'w') as f:
        f.write('')
    filen = f'{main_path}tempo.txt'
    save = True

# Format the arguments for multiprocessing
for i in plane_moon_index:
    injec_argum.append((injec_curve[i], plane_max_numbe, moon_max_numbe,
                        type_orbit_archi, separ_plane_moon, reinj, anima_path))

if __name__ == '__main__':
    with multiprocess.Pool(cpu_number) as pool:
        for i in range(len(curve_save_index) - 1):
            injec_curve[
                curve_save_index[i]:curve_save_index[i + 1]] = pool.starmap(
                    utils.injec_curve,
                    injec_argum[curve_save_index[i]:curve_save_index[i + 1]])
            if save:
                # Remove previous version
                os.remove(filen)
                # Save new version
                filen = f'{xom_data_path}injec_curve/injec_curve-parti-{int(time.time())}.pkl'
                with open(filen, 'wb') as f:
                    pkl.dump(injec_curve, f)

        pool.close()
        pool.join()

if save:
    # Rename the final injected curve
    os.rename(
        filen,
        f'{xom_data_path}injec_curve/injec_curve-full-{int(time.time())}.pkl')

injec_curve = np.array(injec_curve)

ValueError: cannot copy sequence with size 18 to array axis with dimension 92

In [13]:
curve_save_index

[0, 100, 200, 300, 400, 500, 600, 518]

In [12]:
utils.send_task_comple_email('Moons and Planets Injected')

### Detrending

### Cutting, Interpolating, and Padding

In [ ]:
# If detrending is not to be used
detre_curve = np.copy(injec_curve)
for i in range(len(detre_curve)):
    detre_curve[i, -1, 1] = detre_curve[i, -1, 1].copy()

In [ ]:
stand_lengt = 1901
min_lengt = 600
max_time_gap = 10  # mins
caden = 2  # mins
spln_type = 'cubic'

list_cuts = []
for i in range(len(detre_curve)):
    pre_cut = np.copy(detre_curve[i])
    pre_cut[-1, 1] = pre_cut[-1, 1].copy()
    # Insert 'inter_spot' for later interpolation
    pre_cut = utils.inser_inter_spot(pre_cut, max_time_gap, caden)
    # Cut curve and assign cut times to the detrended curve
    pre_cut = utils.cut_curve(pre_cut, max_time_gap, min_lengt, stand_lengt)
    # Cut number
    cut_numbe = 0
    # Find used cut times
    cut_times = []
    for curre_cut in pre_cut:
        # Remove all cuts of only [0,0]
        if curre_cut[-2, 0]: 
            # Interpolate each cut
            cut = utils.inter_curve(curre_cut, spln_type)
            # Assign feature values
            cut[-1, 1]['plane_moon_cut_injec'] = False
            cut[-1, 1]['plane_cut_injec'] = False
            if cut[-1, 1]['type_orbit_archi'] == 'planmoon':
                cut[-1, 1]['plane_moon_cut_injec'] = utils.retur_curve_injec_statu(cut)
            elif cut[-1, 1]['type_orbit_archi'] == 'plan':
                cut[-1, 1]['plane_cut_injec'] = utils.retur_curve_injec_statu(cut)
            cut[-1, 1]['cut_trans'] = cut[-1, 1]['plane_moon_cut_injec'] or cut[-1, 1]['plane_cut_injec']
            cut[-1, 1]['cut_numbe'] = cut_numbe
            cut[-1, 1]['inter_type'] = spln_type
            # Assign cut to list containing the rest of the cuts
            list_cuts.append(cut.tolist())
            # Increase cut number
            cut_numbe += 1
            cut_times.append([cut[utils.find_start(cut), 0], cut[-2, 0]])
    # Assign cut times to the corresponding light, injected, and detrended curve
    light_curve[i, -1, 1]['cut_times'] = cut_times
    injec_curve[i, -1, 1]['cut_times'] = cut_times
    detre_curve[i, -1, 1]['cut_times'] = cut_times
    
padde_cuts = tf.keras.preprocessing.sequence.pad_sequences(list_cuts,
                                                           maxlen=stand_lengt,
                                                           dtype=object)

In [ ]:
# Save data
detre_status = 'detre-' if padde_cuts[0, -1,
                                     1]['detre_time_scale'] else 'not_detre-'
print(f'Detrended Status: {detre_status[:-1]}')
padde_cuts_path = f'{xom_data_path}padde_cuts/padde_cuts-\
{detre_status}{from_secto}_{to_secto}{compl_set}-{int(time.time())}.pkl'
with open(padde_cuts_path, 'wb') as f:
    pkl.dump(padde_cuts, f)
print(f'Padded Cuts Path: {padde_cuts_path}')

In [ ]:
utils.send_task_comple_email('Injection and Cutting')

### Formatting

In [29]:
forma = \
'''
# %%capture

# Number of cpus
cpu_number = 5

# Set up save flag
save = False

# Planet injection setup
plane_max_numbe = 1
moon_max_numbe = 0
type_orbit_archi = 'plan'

injec_argum = []

for i in plane_index:
    injec_argum.append((injec_curve[i], plane_max_numbe, moon_max_numbe, type_orbit_archi))

# Planet and moon injection setup
plane_max_numbe = 1
moon_max_numbe = 1
type_orbit_archi = 'planmoon'
separ_plane_moon = False
reinj = False
anima_path = None
    
# Setup initial save file
if len(curve_save_index) > 2:
    with open(f'{main_path}tempo.txt', 'w') as f:
        f.write('')
    filen = f'{main_path}tempo.txt'
    save = True

# Format the arguments for multiprocessing
for i in plane_moon_index:
    injec_argum.append((injec_curve[i], plane_max_numbe, moon_max_numbe, type_orbit_archi, separ_plane_moon, reinj, anima_path))

if __name__=='__main__':
    with multiprocess.Pool(cpu_number) as pool:
        for i in range(len(curve_save_index) - 1):
            injec_curve[curve_save_index[i]: curve_save_index[i+1]] = pool.starmap(utils.injec_curve, injec_argum[curve_save_index[i]: curve_save_index[i+1]])
            if save:
                # Remove previous version
                os.remove(filen)
                # Save new version
                filen = f'{xom_data_path}injec_curve/injec_curve-parti-{int(time.time())}.pkl'
                with open(filen, 'wb') as f:
                    pkl.dump(injec_curve, f)
                
        pool.close()
        pool.join()
        
if save:
    # Rename the final injected curve
    os.rename(filen, f'{xom_data_path}injec_curve/injec_curve-full-{int(time.time())}.pkl')
        
injec_curve = np.array(injec_curve)
'''
forma_done = FormatCode(forma, style_config='PEP8')
print(forma_done[0])

# %%capture

# Number of cpus
cpu_number = 5

# Set up save flag
save = False

# Planet injection setup
plane_max_numbe = 1
moon_max_numbe = 0
type_orbit_archi = 'plan'

injec_argum = []

for i in plane_index:
    injec_argum.append(
        (injec_curve[i], plane_max_numbe, moon_max_numbe, type_orbit_archi))

# Planet and moon injection setup
plane_max_numbe = 1
moon_max_numbe = 1
type_orbit_archi = 'planmoon'
separ_plane_moon = False
reinj = False
anima_path = None

# Setup initial save file
if len(curve_save_index) > 2:
    with open(f'{main_path}tempo.txt', 'w') as f:
        f.write('')
    filen = f'{main_path}tempo.txt'
    save = True

# Format the arguments for multiprocessing
for i in plane_moon_index:
    injec_argum.append((injec_curve[i], plane_max_numbe, moon_max_numbe,
                        type_orbit_archi, separ_plane_moon, reinj, anima_path))

if __name__ == '__main__':
    with multiprocess.Pool(cpu_number) as pool:
        for i in range(len(curve_save_index) - 1)